In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
import os
from tqdm import tqdm  # Import tqdm for progress bars


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Data Transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_random = transforms.Compose([
    transforms.RandomRotation(10),  # Rotate images by up to 10 degrees
    transforms.RandomResizedCrop(128, scale=(0.9, 1.1)),  # Zoom augmentation
    transforms.RandomAffine(0, translate=(0.1, 0.1)),  # Width and height shift
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load Dataset
# train_dataset = torchvision.datasets.ImageFolder(root="Dataset/Train", transform=transform)
# val_dataset = torchvision.datasets.ImageFolder(root="Dataset/Val", transform=transform)

# Create DataLoaders with batching
# batch_size = 32
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Define CNN Model
class GenderClassifier(nn.Module):
    def __init__(self):
        super(GenderClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x


In [ ]:
# Initialize Model, Loss, and Optimizer
model = GenderClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.95 ** epoch)


In [ ]:
nets = 15
models = [GenderClassifier().to(device) for _ in range(nets)]
criterion = nn.BCELoss()
optimizers = [optim.Adam(model.parameters(), lr=0.001) for model in models]
schedulers = [optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.95 ** epoch) for optimizer in optimizers]

In [ ]:
# Train Model with tqdm Progress Bar
num_epochs = 25

for j in range(nets):
    model = models[j]
    optimizer = optimizers[j]
    scheduler = schedulers[j]

    # Load Dataset
    train_dataset = torchvision.datasets.ImageFolder(root="Dataset/Train", transform=transform_random)
    # Create DataLoaders with batching
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    for epoch in range(num_epochs):

        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            progress_bar.set_postfix(loss=running_loss / len(train_loader))  # Update tqdm bar
        
        scheduler.step()

        print(f"CNN {j + 1}: Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")


In [ ]:
val_dataset = torchvision.datasets.ImageFolder(root="Dataset/Val", transform=transform)
val_loader = DataLoader(val_dataset)

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
        outputs = 0
        for j in range(nets):
            outputs += model(images)
        predicted = (outputs > nets / 2).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

In [ ]:
import cv2
import matplotlib.pyplot as plt
img = Image.open("Dataset/Test/000067.jpg").convert("RGB")
# plt.imshow(cv2.cvtColor(cv2.imread("Dataset/Test/000067.jpg"), cv2.COLOR_BGR2RGB))
# plt.show()
# img.show()
img = transform(img).unsqueeze(0).to(device) 
output = model(img)
print(output.item())  # Output is a tensor, convert to float for display

In [ ]:
test_path = "Dataset/Test"

# Load images
test_images = []
img_id = []
for img_name in os.listdir(test_path):
    img_path = os.path.join(test_path, img_name)
    if img_name.endswith(('.jpg', '.png', '.jpeg')):  # Filter image files
        img = Image.open(img_path).convert("RGB")
        img = transform(img)
        test_images.append(img)
        # remove file extension for ID
        img_id.append(os.path.splitext(img_name)[0])

# Convert to tensor batch
test_images = torch.stack(test_images)
print(test_images.shape)  # Shape: (num_images, channels, height, width)

test_images = test_images.to(device)
outputs = model(test_images)
predicted = (outputs < 0.5).int()

print(predicted.shape)  # Shape: (num_images, 1)

In [ ]:
# export to csv with 2 column ID and label
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "ID": img_id,
    "Label": predicted.cpu().numpy().flatten()
})
df.to_csv("submission.csv", index=False)
print("Submission file created: submission.csv")
